# ELLE - Elastic Learning

## Basics (can be ignored)

### Imports

In [26]:
import numpy as np
import theano
from numpy import random as rng
from scipy.io import savemat
import matplotlib.pyplot as plt
import matplotlib.pyplot as P
import matplotlib.ticker as MT
import matplotlib.cm as CM
import cPickle
import gzip
import os
from sklearn.metrics import precision_recall_fscore_support

PATH_DATA_ROOT = r"../data"

### Libs

In [14]:
def linear(x):
    return x

def dlinear(x):
    return np.ones(x.shape)

def tanh(x):
    return np.tanh(x)

def dtanh(x):
    tan = tanh(x)
    return (1. - tan**2.)

def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def dsigmoid(x):
    sig = sigmoid(x)
    return sig*(1.-sig)

def softmax(x):
    e = np.exp(x)
    return e / np.sum(e)#[:, np.newaxis]

def dsoftmax(x):
    """
    NOTE: When computing the gradient of a combination of softmax output and crossentropy error,
    use :func:`~slowlearning.dsce_error` instead.
    Computing :func:`~slowlearning.dce_error` and :func:`~slowlearning.dsoftmax` separately is computationally very inefficient
    :param x:
    :return: Jacobean matrix. partial derivatives of all outputs :func:`~slowlearning.softmax` with respect to all inputs x
    """
    p = softmax(x)
    Ds = -np.outer(p, p)
    di = np.diag_indices(len(x))
    Ds[di] = p-p**2.
    return Ds

def rec_error(p, y):
    return 0.5 * np.sum((p - y)**2.)

def drec_error(p, y):
    return (p - y)

def dlinrec_error(x, y):
    return x - y

def ceb_error(p, y):
    eps = 1e-10
    return - np.sum(y * np.log(p + eps) + (1. - y) * np.log(1. - p + eps))

def dceb_error(p, y):
    return - y * 1. / p + (1. - y) / (1. - p)

def dlogceb_error(x, y):
    p = sigmoid(x)
    return - y * (1. - p) + (1. - y) * p

def cem_error(p, y):
    eps = 1e-10
    return - np.sum(y * np.log(p + eps))

def dcem_error(p, y):
    return - y * 1. / p

def dsofcem_error(x, y):
    return softmax(x)-y

### Utils

In [15]:
class T_Func_Type:
    TANH = "tanh"
    LOGISTIC = "logistic"


class T_OutFunc_Type:
    SOFTMAX = "softmax"
    LINEAR = "linear" # linear output activation function in fact corresponds to gaussian output variables
    LOGISTIC = "logistic"


class T_ErrFunc_Type:
    RECONSTRUCTION = "reconstruction"
    CROSS_ENTROPY_BINOMIAL = "cross-entropy-binomial"
    CROSS_ENTROPY_MULTINOMIAL = "cross-entropy-multinomial"


class Function(object):
    def __init__(self, f, df):
        self.f = f
        self.df = df

actFuncs = {T_Func_Type.TANH: Function(tanh, dtanh),
            T_Func_Type.LOGISTIC: Function(sigmoid, dsigmoid)}

outFuncs = {T_OutFunc_Type.SOFTMAX: Function(softmax, dsoftmax),
            T_OutFunc_Type.LINEAR: Function(linear, dlinear),
            T_OutFunc_Type.LOGISTIC: Function(sigmoid, dsigmoid)}

errFuncs = {T_ErrFunc_Type.RECONSTRUCTION: Function(rec_error, drec_error),
            T_ErrFunc_Type.CROSS_ENTROPY_BINOMIAL: Function(ceb_error, dceb_error),
            T_ErrFunc_Type.CROSS_ENTROPY_MULTINOMIAL: Function(cem_error, dcem_error)}

In [16]:
class Utils:
    @staticmethod
    def code1ofK(labels, K):
        KcodedLabels = []
        for l in labels:
            codedK = np.zeros(shape=(K,))
            codedK[int(l)] = 1.
            KcodedLabels.append(codedK)
        return KcodedLabels
    
    @staticmethod
    def load_mnist():
        with gzip.open(r"../data/mnist.pkl.gz", 'rb') as f:
            tr,te,vl = cPickle.load(f)
        return tr, te, vl

    @staticmethod
    def validate(goldLabels, predictedLabels):
        (pre, rec, f1, sup) = precision_recall_fscore_support(goldLabels, predictedLabels, beta=1.0, labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None)
        np.set_printoptions(precision=3)
        print pre
        print rec
        print f1
        print sup
        return (pre, rec, f1, sup)

    @staticmethod
    def visualize_weights(weights, panel_shape, tile_size):

        def scale(x):
            eps = 1e-8
            x = x.copy()
            x -= x.min()
            x *= 1.0 / (x.max() + eps)
            return 255.0*x

        margin_y = np.zeros(tile_size[1])
        margin_x = np.zeros((tile_size[0] + 1) * panel_shape[0])
        image = margin_x.copy()

        for y in range(panel_shape[1]):
            tmp = np.hstack( [ np.c_[ scale( x.reshape(tile_size) ), margin_y ] for x in weights[y*panel_shape[0]:(y+1)*panel_shape[0]]])
            tmp = np.vstack([tmp, margin_x])
            image = np.vstack([image, tmp])

        img = Image.fromarray(image)
        img = img.convert('RGB')
        return img


class OutputModel(object):
    def __init__(self, outFuncType, errFuncType):
        self.p = outFuncs[outFuncType]
        self.E = errFuncs[errFuncType]
        if outFuncType == T_OutFunc_Type.SOFTMAX and errFuncType == T_ErrFunc_Type.CROSS_ENTROPY_MULTINOMIAL:
            self.dEdx = dsofcem_error
        elif outFuncType == T_OutFunc_Type.LOGISTIC and errFuncType == T_ErrFunc_Type.CROSS_ENTROPY_BINOMIAL:
            self.dEdx = dlogceb_error
        elif outFuncType == T_OutFunc_Type.LINEAR and errFuncType == T_ErrFunc_Type.RECONSTRUCTION:
            self.dEdx = dlinrec_error
        else:
            self.dEdx = "composite"

    def cost_out_grad(self, x, y):
        out = self.p.f(x)
        if self.dEdx == "composite":
            return (self.E.f(out, y), out, self.E.df(self.p.f(x), y) * self.p.df(x))
        else:
            return (self.E.f(out, y), out, self.dEdx(x, y))

## Theory

### Softmax and Cross-Entropy

We use the softmax output function to compute class probabilites $p_k$ for each example. In a supervised $K$-multi-class classification setting, the probability for an example $x$ to belong to class $C_k$ is given a-priori by $y_k = P(C_k|x)$. We measure the deviation of predicted values $p_k$ from the target values $y_k$ by means of the cross-entropy error $L(x,y)$. The derivatives of the cross-entropy error with respect to the inputs of the softmax are given below. For implementation purposes, it's much more efficient to pre-compute the derivative of $\frac{\partial L(softmax(x),y)}{\partial x}$ instead of computing and multiplying the gradient $\frac{\partial L(p, y)}{\partial p}$ with the Jacobean $\frac{\partial softmax(x)}{\partial{x}}$

$$p_k(\mathbf{x}) = softmax_k(\mathbf{x}) = \frac{e^{x_k}}{\sum_i e^{x_i}}$$
if $k = i$:
\begin{align}
\frac{\partial p_k}{\partial x_i} &= \frac{e^{x_k}}{\sum_i e^{x_i}} + e^{x_k} \cdot - \frac{1}{(\sum_i e^{x_i})^2} \cdot e^{x_i} \\
 &= \frac{e^{x_k}}{\sum_i e^{x_i}} - \frac{e^{x_k}}{\sum_i e^{x_i}} \cdot \frac{e^{x_i}}{\sum_i e^{x_i}} \\
 &= p_k - p_k p_i\\
 &= p_k (1 - p_i) 
\end{align}

if $k \ne i$:

\begin{align}
\frac{\partial p_k}{\partial x_i} &= e^{x_k} \cdot - \frac{1}{(\sum_i e^{x_i})^2} \cdot e^{x_i} \\
 &= -p_k p_i
\end{align}
$L(\mathbf{x}, \mathbf{y}) = - \sum_k y_k \cdot \log p_k(\mathbf{x})$
\begin{align}
\frac{\partial L}{\partial x_i} &= - \sum_k y_k \cdot \frac{1}{p_k} \cdot \frac{\partial p_k}{\partial x_i} \\
 &= - y_i (1 - p_i) + \sum_{k \ne i} y_k p_i \\
 &= - y_i + \sum_k y_k p_i \\
 &= - y_i + p_i \cdot \underbrace{\sum_k y_k}_{= 1} \\
 &= p_i - y_i
\end{align}

### Codes and Decision Boundaries

each Bit represents a linear decision boundary, partitionning the space into 2 regions. Each extra Bit again partitions each existing region into another 2 regions -> d-Bits = 2^d regions (states). Density-Estimation. In this context, figure out the relationship of: Entropy, Correlation, Frequency, Dependency of Random Variables. Buzzwords: Correlation-based learning, slow feature analysis, manifold-learning.

## Autoencoder

In [17]:
class Autoencoder(object):
    def __init__(self, nvis=100, nhid=50, eta=0.1, actfunc=actFuncs[T_Func_Type.TANH]):

        self.visible_size = nvis
        self.hidden_size = nhid

        self.W = np.asarray(rng.uniform(low=-4 * np.sqrt(6. / (self.hidden_size + self.visible_size)), high=4 * np.sqrt(6. / (self.hidden_size + self.visible_size)), size=(self.hidden_size, self.visible_size)), dtype=theano.config.floatX)
        self.b1 = np.zeros(shape=(self.hidden_size,), dtype=theano.config.floatX)
        self.b2 = np.zeros(shape=(self.visible_size,), dtype=theano.config.floatX)

        self.actfunc = actfunc
        self.eta = eta

    def _encode(self, x):
        return np.dot(self.W, x) + self.b1

    def _decode(self, h):
        return np.dot(self.W.T, h) + self.b2

    def _get_rec_err(self, x, z):
        return 0.5 * np.sum((x-z)**2.)

    def _get_ce_err(self, x, z):
        eps = 1e-10
        return - np.sum((x * np.log(z + eps) + (1.-x) * np.log(1.-z + eps)))

    def init_supervised(self, nout):
        self.output_size = nout
        self.Wlabel = np.asarray(rng.uniform(low=-4 * np.sqrt(6. / (self.output_size + self.hidden_size)), high=4 * np.sqrt(6. / (self.output_size + self.hidden_size)), size=(self.output_size, self.hidden_size)), dtype=theano.config.floatX)
        self.blabel = np.zeros(shape=(self.output_size,), dtype=theano.config.floatX)
        self.OutModel = OutputModel(T_OutFunc_Type.SOFTMAX, T_ErrFunc_Type.CROSS_ENTROPY_MULTINOMIAL)

    def get_cost_grad(self, batch):

        cost = 0.
        g_W = np.zeros(self.W.shape)
        g_b1 = np.zeros(self.b1.shape)
        g_b2 = np.zeros(self.b2.shape)

        for x in batch:
            a = self._encode(x)
            h = self.actfunc.f(a)
            p = self._decode(h)

            cost += rec_error(p, x)

            deltaOut = drec_error(p, x)

            g_W += np.outer(deltaOut, h).T
            g_b2 += deltaOut

            deltaHidden = np.dot(self.W, deltaOut) * self.actfunc.df(a)
            g_W += np.outer(deltaHidden, x)
            g_b1 += deltaHidden

        cost /= len(batch)
        g_W /= len(batch)
        g_b1 /= len(batch)
        g_b2 /= len(batch)

        return cost, g_W, g_b1, g_b2

    def get_supcost_grad(self, batch, targets):

        batch_cost = 0.
        g_W = np.zeros(self.W.shape)
        g_b1 = np.zeros(self.b1.shape)
        g_Wlabel = np.zeros(self.Wlabel.shape)
        g_blabel = np.zeros(self.blabel.shape)

        for i, x in enumerate(batch):
            a = self._encode(x)
            h = self.actfunc.f(a)
            o = np.dot(self.Wlabel, h) + self.blabel
            (cost, out, grad) = self.OutModel.cost_out_grad(o, targets[i])
            batch_cost += cost

            deltaOut = grad
            g_Wlabel += np.outer(deltaOut, h)
            g_blabel += deltaOut

            deltaHidden = np.dot(self.Wlabel.T, deltaOut) * self.actfunc.df(a)
            g_W += np.outer(deltaHidden, x)
            g_b1 += deltaHidden

        batch_cost /= len(batch)
        g_W /= len(batch)
        g_b1 /= len(batch)
        g_Wlabel /= len(batch)
        g_blabel /= len(batch)

        return batch_cost, g_W, g_b1, g_Wlabel, g_blabel

    def train(self, data, epochs=2, batch_size=20, freeIndex=0):

        batch_num = len(data) / batch_size

        for epoch in xrange(epochs):
            total_cost = 0.
            self.eta *= 0.99
            for i in xrange(batch_num):
                batch = data[i*batch_size : (i+1)*batch_size]
                (cost, g_W, g_b1, g_b2) = self.get_cost_grad(batch)
                total_cost += cost
                self.W[freeIndex] -= self.eta * g_W[freeIndex]
                self.b1[freeIndex] -= self.eta * g_b1[freeIndex]
                self.b2 -= self.eta * g_b2

            print "Epoch: %d" % epoch
            print (1. / batch_num) * total_cost

    def trainSupervised(self, data, targets, epochs=10, batch_size=20):

        batch_num = len(data) / batch_size

        for epoch in xrange(epochs):
            total_cost = 0.
            self.eta = 0.99
            for batchIdx in xrange(batch_num):
                batch = data[batchIdx * batch_size : (batchIdx+1) * batch_size]
                batch_targets = targets[batchIdx * batch_size : (batchIdx+1) * batch_size]
                (cost, g_W, g_b1, g_Wlabel, g_blabel) = self.get_supcost_grad(batch, batch_targets)
                total_cost += cost
                self.W -= self.eta * g_W
                self.b1 -= self.eta * g_b1
                self.Wlabel -= self.eta * g_Wlabel
                self.blabel -= self.eta * g_blabel

            print "Epoch: %d" % epoch
            print (1. / batch_num) * total_cost

    def predict(self, data):

        if self.OutModel is None:
            return None

        predictedTargets = []
        predictedLabels = []

        for x in data:
            a = self._encode(x)
            h = self.actfunc.f(a)
            o = np.dot(self.Wlabel, h) + self.blabel
            pred = self.OutModel.p.f(o)
            idx = np.argmax(pred)
            predictedTargets.append(pred)
            predictedLabels.append(idx)

        return (predictedTargets, predictedLabels)

    def visualize_filters(self, name=None):
        tile_size = (int(np.sqrt(self.W[0].size)), int(np.sqrt(self.W[0].size)))
        panel_shape = (int(len(self.W)/11)+1, len(self.W) % 11)
        img = Utils.visualize_weights(self.W, panel_shape, tile_size)
        if name is not None:
            img.save(name+".png", "PNG")
        img.show()

## Elastic Learning

In [18]:
class ElasticLearning(object):
    def __init__(self, data, labels):
        self.data = data
        self.targets = labels
        self.visibles = len(data[0])

    def findCode(self, codeLimit=10):

        self.tmpW = None
        self.tmpb1 = None
        self.tmpb2 = None

        bitRange = range(1, codeLimit+1)

        for (idx, codeSize) in enumerate(bitRange):
            fixedRange = range(0, idx)
            print "CodeSize: %d/%d" % (codeSize, codeLimit)
            self.ae = Autoencoder(nvis=self.visibles, nhid=codeSize)
            if codeSize == 1:
                self.ae.train(self.data, epochs=2, freeIndex=idx)
                self.tmpW = self.ae.W
                self.tmpb1 = self.ae.b1
                self.tmpb2 = self.ae.b2
            else:
                self.ae.W[fixedRange] = self.tmpW
                self.ae.b1[fixedRange] = self.tmpb1
                self.ae.b2 = self.tmpb2
                self.ae.train(self.data, epochs=2, freeIndex=idx)
                self.tmpW = self.ae.W
                self.tmpb1 = self.ae.b1
                self.tmpb2 = self.ae.b2

        self.dump_model()

    def finetune(self):
        self.ae.init_supervised(len(self.targets[0]))
        self.ae.trainSupervised(self.data, self.targets, epochs=10, batch_size=20)

    def dump_model(self):
        cPickle.dump(self.ae, open(PATH_DATA_ROOT+r"/model.pkl", "w"))

    def load_model(self):
        self.ae = cPickle.load(open(PATH_DATA_ROOT+r"/model.pkl", "r"))
        return self.ae

# Experiments on MNIST

Our first experiment is a very basic one. We compare the elastic learning and the compound learning method on a single hidden layer network. 
Compound Learning:
This is the standard backpropagation algorithm. It trains all nodes in the hidden layer simultaneously as a bunch of composed functions. As usual, the hidden layer is pre-trained via reconstruction error. After the pre-training phase (upper limit epoch=limit or $\Delta Error \lt \epsilon$) we fine-tune the weights with respect to the supervised error criterion.

In [24]:
import time

train_data, test_data, valid_data = Utils.load_mnist()
K = 10

print "Elastic Learning Algorithm:"
ella = ElasticLearning(train_data[0], Utils.code1ofK(train_data[1], 10))
strttime = time.time()
ella.findCode()
endtime = time.time()
print "pre-training took %.2f s" % (endtime-strttime)
#ella.ae.visualize_filters("elastic-pretrained")
print "Fine-Tuning with label information:"
ella.finetune()
#ella.ae.visualize_filters("elastic-finetuned")
print "Test Model:"
(pred_targets, pred_labels) = ella.ae.predict(test_data[0])
(pre, rec, f1, sup) = Utils.validate(test_data[1], pred_labels)
elastic = f1

print "Compound Learning Algorithm:"
ae = Autoencoder(nvis=784, nhid=10, eta=0.1)
strttime = time.time()
ae.train(train_data[0], epochs=20, batch_size=20, freeIndex=range(10))
endtime = time.time()
print "pre-training took %.2f s" % (endtime-strttime)
#ae.visualize_filters("compound-pretrained")
print "Fine-Tuning with Label Information"
ae.init_supervised(K)
ae.trainSupervised(train_data[0], Utils.code1ofK(train_data[1], K), epochs=10, batch_size=20)
#ae.visualize_filters("compound-finetuned")
print "Test Model:"
(pred_targets, pred_labels) = ae.predict(test_data[0])
(pre, rec, f1, sup) = Utils.validate(test_data[1], pred_labels)
compound = f1

Elastic Learning Algorithm:
CodeSize: 1/10
Epoch: 0
26.1463134712
Epoch: 1
26.1119105185
CodeSize: 2/10
Epoch: 0
24.5531930612
Epoch: 1
24.2517222973
CodeSize: 3/10
Epoch: 0
24.3191618102
Epoch: 1
22.8561457221
CodeSize: 4/10
Epoch: 0
21.7154252466
Epoch: 1
21.547304021
CodeSize: 5/10
Epoch: 0
20.7092914072
Epoch: 1
20.4933635909
CodeSize: 6/10
Epoch: 0
19.6232492431
Epoch: 1
19.5253882057
CodeSize: 7/10
Epoch: 0
19.2772963155
Epoch: 1
18.6199839685
CodeSize: 8/10
Epoch: 0
18.1329145595
Epoch: 1
17.8763162793
CodeSize: 9/10
Epoch: 0
17.3101358058
Epoch: 1
17.2471438195
CodeSize: 10/10
Epoch: 0
16.7070933705
Epoch: 1
16.6572121649
pre-training took 67.55 s
Fine-Tuning with label information:
Epoch: 0
0.84470593266
Epoch: 1
0.594259286759
Epoch: 2
0.54815990559
Epoch: 3
0.467092668343
Epoch: 4
0.444181307399
Epoch: 5
0.42732942827
Epoch: 6
0.412641888515
Epoch: 7
0.409795384781
Epoch: 8
0.40255350665
Epoch: 9
0.403987568678
Test Model:
[ 0.94   0.894  0.949  0.884  0.904  0.797  0.927  0

In [27]:
n_groups = K
fig, ax = plt.subplots()

index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, elastic, bar_width,
                     alpha=opacity,
                     color='b',
                     error_kw=error_config,
                     label='Elastic')

rects2 = plt.bar(index + bar_width, compound, bar_width,
                     alpha=opacity,
                     color='r',
                     error_kw=error_config,
                     label='Compound')

plt.xlabel('Digit Group')
plt.ylabel('F1-Score')
plt.title('F1-Scores on MNIST by Digit Groups and Method')
plt.xticks(index + bar_width, ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))
plt.legend()

plt.tight_layout()
plt.show()

/Users/johannesjurgovsky/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
